# Order Nakeds

In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = "NSE"

# Set the root
from from_root import from_root
ROOT = from_root()

import pandas as pd
from loguru import logger

pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

from pathlib import Path
import sys

# Add `src` and ROOT to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path

if "src" not in Path.cwd().parts:
    src_path = str(Path(get_path("purelib")) / "_src.pth")
    with open(src_path, "w") as f:
        f.write(str(ROOT / "src\n"))
        f.write(str(ROOT))
        if str(ROOT) not in sys.path:
            sys.path.insert(1, str(ROOT))

# Start the Jupyter loop
from ib_async import util

util.startLoop()

logger.add(sink=ROOT / "log" / "ztest.log", mode="w")

1

## Imports

In [2]:
from utils import handle_raws, get_pickle, load_config, arrange_orders, pickle_me, get_file_age, yes_or_no
from datetime import datetime
from ib_async import IB
from ibfuncs import get_open_orders, quick_pf, place_orders, make_ib_orders

## Set constants

In [3]:
config = load_config()
port = config.get('PORT')
MARGINPERORDER = config.get('MARGINPERORDER')

## Handle Raws

In [4]:
# Consolidate raw files to df_nakeds.pkl
handle_raws()

Do you want to archive raw nakeds? (y/n):  y


2024-08-02 01:47:31.322 | INFO     | utils:delete_files:155 - Deleted: C:\Users\kashi\python\nse\data\raw\earliest_nakeds1.pkl
2024-08-02 01:47:31.324 | INFO     | utils:delete_files:155 - Deleted: C:\Users\kashi\python\nse\data\raw\earliest_nakeds10.pkl
2024-08-02 01:47:31.325 | INFO     | utils:delete_files:155 - Deleted: C:\Users\kashi\python\nse\data\raw\earliest_nakeds2.pkl
2024-08-02 01:47:31.328 | INFO     | utils:delete_files:155 - Deleted: C:\Users\kashi\python\nse\data\raw\earliest_nakeds3.pkl
2024-08-02 01:47:31.330 | INFO     | utils:delete_files:155 - Deleted: C:\Users\kashi\python\nse\data\raw\earliest_nakeds4.pkl
2024-08-02 01:47:31.333 | INFO     | utils:delete_files:155 - Deleted: C:\Users\kashi\python\nse\data\raw\earliest_nakeds5.pkl
2024-08-02 01:47:31.335 | INFO     | utils:delete_files:155 - Deleted: C:\Users\kashi\python\nse\data\raw\earliest_nakeds6.pkl
2024-08-02 01:47:31.336 | INFO     | utils:delete_files:155 - Deleted: C:\Users\kashi\python\nse\data\raw\earl

In [5]:
file_path = ROOT / 'data' / 'df_nakeds.pkl'

def how_many_days_old(file_path) -> float:
    """Gets the file's age in days"""
    file_age = get_file_age(file_path=file_path)
    
    seconds_in_a_day = 86400
    file_age_in_days = file_age.td.total_seconds() / seconds_in_a_day if file_age else 0
    
    return file_age_in_days

In [6]:
## Check the age of df_pickles, before ordering

In [9]:
txt = f"df_nakeds.pkl is {how_many_days_old(file_path): 0.2f}. Want to load?"
ans = yes_or_no(txt)

if ans:
    df_opts = get_pickle(file_path)
    print('\n\n')
    print(df_opts.drop(columns=['nse_symbol', 'instrument', 'contract', 'expiry']).head())
else:
    print('Bye!!!')

df_nakeds.pkl is  0.00. Want to load? (y/n):  y





  ib_symbol  undPrice  safe_strike right  strike    dte    hv    iv   lot  \
0  AARTIIND    741.65          856     C   740.0  27.73  0.41  0.31  1000   
1  AARTIIND    741.65          619     P   750.0  27.73  0.41  0.36  1000   
2  AARTIIND    741.65          859     C   750.0  27.73  0.41  0.32  1000   
3  AARTIIND    741.65          625     P   740.0  27.73  0.41  0.35  1000   
4  AARTIIND    741.65          860     C   760.0  27.73  0.41  0.32  1000   

   price   sdev  intrinsic  bsPrice  comm    margin  xPrice    rom  
0  29.35  64.02      116.0    28.18  20.0  47115.57  145.35  40.61  
1  31.10  74.16      131.0    32.05  20.0  46340.68  163.05  46.32  
2  25.10  65.74      109.0    24.04  20.0  42180.56  134.10  41.85  
3  24.65  70.60      115.0    25.49  20.0  41912.95  140.50  44.13  
4  20.90  66.06      100.0    19.94  20.0  37041.30  120.90  42.97  


## check open orders

In [10]:
# check open orders
with IB().connect(port=port, clientId=10) as ib:
    dfo = get_open_orders(ib)
    dfp = quick_pf(ib)
    

In [11]:
if not dfo.empty:
    remove_opens = set(dfo.symbol.to_list())
else:
    remove_opens = set()

In [12]:
# make a list of symbols to be removed from df_opts

if not dfp.empty:
    remove_positions = set(dfp.symbol.to_list())
else:
    remove_positions = set()

remove_ib_syms = remove_opens | remove_positions

# get the target options to plant
dft = df_opts[~df_opts.ib_symbol.isin(remove_ib_syms)].reset_index(drop=True)

In [13]:
len(dft)

0

## Arrange and make orders

## PLACE THE ORDER

# Archive the orders into `xn_history`

In [8]:
from ibfuncs import order_nakeds
order_nakeds(df_opts=df_opts, port=port)

No raw files to archive


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.31it/s]
2024-08-02 01:48:38.754 | INFO     | ibfuncs:order_nakeds:294 - Successfully placed 88 orders


[Trade(contract=Option(conId=712949612, symbol='ULTRACEMC', lastTradeDateOrContractMonth='20240829', strike=11500.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='ULTRACEMCO24AUG11500PE', tradingClass='ULTRACEMCO'), order=LimitOrder(orderId=92578, clientId=10, permId=1654946750, action='SELL', totalQuantity=100.0, lmtPrice=818.25, auxPrice=0.0), orderStatus=OrderStatus(orderId=92578, status='PreSubmitted', filled=0.0, remaining=100.0, avgFillPrice=0.0, permId=1654946750, parentId=0, lastFillPrice=0.0, clientId=10, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 8, 1, 17, 48, 35, 705791, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 8, 1, 17, 48, 36, 582250, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0)], advancedError=''),
 Trade(contract=Option(conId=707140319, symbol='VOLTAS', lastTradeDateOrContractMonth='202